In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import pyodbc
import urllib
import joblib
import json
from sqlalchemy import create_engine 
from datetime import datetime, date
import sys
pd.set_option('display.max_columns', None)

### Get the two seperate IBVStatusID lists + performances table into dfs 

In [ ]:
# load the csvs into dfs 
empty_model_requests = pd.read_csv("/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_data/empty_model_requests.csv")
non_empty_model_requests = pd.read_csv("/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_data/non_empty_model_requests.csv")

,CModelLogID,ModelName,ApplicationID,PortfolioID,IBVStatusID,DateProcessed,ModelRequest,ModelResponse,ProcessingTime,ApplicationDate,abs_diff_seconds
0,1315,ibv,0,0,757,2025-05-10 12:42:07.290,"{""asOfDate"":""2025-05-10"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""ErqbnE1VRNhYZzm0r...",3031,NaN,NaN
1,5456,ibv,0,0,1258,2025-08-27 08:51:35.213,"{""asOfDate"":""2025-08-27"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""Z1y3oybRZwsyz97P1...",1563,NaN,NaN
2,4645,ibv,0,0,1382,2025-08-05 11:45:12.307,"{""asOfDate"":""2025-08-05"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""639pvqA83piYK5gjM...",2546,NaN,NaN
3,1756,ibv,0,0,1810,2025-05-22 09:35:23.653,"{""asOfDate"":""2025-05-22"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""AeRBY75BDwUm78Mon...",1562,2025-05-22 09:24:41,642.653
4,2838,ibv,0,0,2334,2025-06-24 12:28:42.160,"{""asOfDate"":""2025-06-24"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""5PeDy5EZ9mtLb6kQD...",2203,2025-06-24 09:29:04,10778.160


In [7]:
from sqlalchemy import create_engine 
import urllib
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 
USE LF_LMSMASTER

drop table if EXISTS #t1
select ibvT.IBVStatusID, A.Application_ID, A.PortfolioID, A.CustomerID, A.ApplicationDate, CustEmail,
case when ApplicationSteps like '%S%' then 1 else 0 end as NewlyScored,
case when A.ApplicationStatus in ('A','P') then 1 else 0 end as Accepted,
case when A.ApplicationStatus in ('A','P') then A.LeadPurchasePrice else 0 end as LeadPurchasePrice,
case when L.LoanStatus not in ('V','W','G','K') and A.ApplicationStatus='J'  then 1 else 0 end as PartialPreApproved,
case when L.LoanStatus not in ('V','W','G','K')  then 1 else 0 end as Originated,
L.LoanID, datediff(year, VW.DOB, A.ApplicationDate) as Age,
case when VW.Frequency in ('B','S') then 'B' else VW.Frequency end as Frequency,
case when L.LoanStatus not in ('V','W','G','K') then L.OriginatedAmount else null end as OriginatedAmount,
LP.Provider_name, OriginationDate, LOANSTATUS,
case when ((L.RenewalLoanId <> '0') or (A.LPCampaign = 'RENEW'))  then 'RENEWAL' when ApplicationSteps not like '%R%' then 'NEW' else 'REPEAT' end as CustType, RenewalLoanId
into #t1
from Application A
left join Loans L on A.Application_ID = L.ApplicationID and A.PortFolioID = L.PortFolioID
left join LeadProvider LP on A.LeadProviderID = LP.LeadProviderID
left join LF_LMS_Logs..VW_ApplicationDump VW on A.APPGUID = VW.APPGUID
--left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.AccountNumber = A.CustomerSSN
left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.ApplicationID = A.Application_ID AND ibvT.PortfolioID= A.PortfolioID
where A.ApplicationDate>'2023-04-30' and ((CustEmail is null) or  (CustEmail not in ( 'josh@concordadvice.com', 'tiffany.speedyservicing@gmail.com','bobby@speedyservicing.com',
'and.kom@example.com', 'and.tor020@example.com', 'and.tor050@example.com', 'and.torrc4@example.com',
'test@dot818.com', 'test@example.com', 'test@gmail.com', 'test@loanscanada.ca', 'test@test.com', 'test2@test.com',
'testerqad@gmail.com', 'testik@test.com', 'testteam@dmaassociatescorp.com', 'tiffany.speedyservicing@gmail.com')))

UPDATE #t1 -- update the renewal loanid from 0 to correct previous loanIDs
SET #t1.RenewalLoanId = B.RenewalLoanId
from #t1 A inner join (select * from
(select A.LoanID, L.LoanID as RenewalLoanId, row_number() over (partition by A.LoanID order by datediff(day, L.OriginationDate, A.OriginationDate) desc) as RN from #t1 A
inner join Application A2 on A.CustomerID = A2.CustomerID and A2.ApplicationDate < A.ApplicationDate
inner join Loans L on A2.Application_ID = L.ApplicationID and A2.PortFolioID = L.PortFolioID and L.OriginationDate < A.OriginationDate and L.LoanStatus not in ('W','V','G','K')
where A.CustType = 'RENEWAL' and A.RenewalLoanId = '0') K where RN=1) B on A.LoanID = B.LoanID

-- Peak #t1 temp table
/* SELECT *
FROM #t1
WHERE IBVStatusID = 1810 */

drop table if EXISTs #t11
select A.Application_ID, A.PortFolioID, A.LoanID, rank() over(partition by A.CustomerID order by OriginationDate asc) as CustSSNLoanNum
into #t11
from #t1 A
where A.Originated=1

-- Peak #t11 temp table
/* SELECT *
FROM #t11 */
--WHERE IBVStatusID = 1810

-- scoring result for PlaidUDW_v1 and NegativeDBModelLP_v1
drop table if EXISTS #t2
select *
into #t2 from
(select A.*, B1.ExtResponse as B1ExtResponse, B2.ExtResponse as B2ExtResponse,
NDScore, IBVScore, NDBand,IBVBand,
row_number() over (partition by A.Application_ID, A.PortfolioID order by case when B2.ExtResponse is not null and B1.ExtResponse <> '' then 1 else 0 end desc,B.ApplicationDate desc) as RN
from #t1 A
left join Application B on A.CustomerID = B.CustomerID and A.ApplicationDate >= B.ApplicationDate and B.ApplicationSteps like '%S%'
left join ScoringPythonResult B1 CROSS APPLY OPENJSON(B1.ExtResponse) WITH (IBVScore int '$.ModelScore', IBVBand int '$.IBVBand')  on B.Application_ID = B1.ApplicationID and B.PortFolioID = B1.PortfolioID and B1.iLabel= 'IBVBand'
left join ScoringPythonResult B2 CROSS APPLY OPENJSON(B2.ExtResponse) WITH (NDScore int '$.ModelScore', NDBand int '$.NDBand') on B.Application_ID = B2.ApplicationID and B.PortFolioID = B2.PortfolioID and B2.iLabel= 'NDBand' ) K where RN=1
ALTER TABLE #t2
DROP COLUMN RN;
--peak #t2 table
/* SELECT TOP 10 *
FROM #t2
WHERE IBVStatusID IS NOT NULL */

/* ================================
   GET FPDFA new logic
   STEP 1: BASE TABLE – FIRST INSTALLMENTS ONLY
   ================================ */
DROP TABLE IF EXISTS #LF_LoanDefault;

SELECT  
      L.LoanID
    , L.ApplicationID AS Application_ID
    , A.APPGUID
    , A.ApplicationDate
    , A.ApplicationSteps
    , L.PortfolioID
    , L.LoanStatus
    , P.InstallmentNumber
    , P.PaymentStatus
    , P.PaymentType
    , P.PaymentMode
    , P.AttemptNo
    , P.TransactionDate
    , P.PaymentID
    , I.InstallmentID
    , I.iPaymentMode
    , I.DueDate
    , I.Status
    , CASE WHEN I.Status = 684 THEN 1 ELSE 0 END AS Pending    -- exclude pendings
INTO #LF_LoanDefault
FROM LF_LMSMaster..Loans L
LEFT JOIN LF_LMSMaster..Payment      P ON P.LoanID      = L.LoanID
LEFT JOIN LF_LMSMaster..Installments I ON I.InstallmentID = P.InstallmentID
LEFT JOIN LF_LMSMaster..Application  A 
       ON A.PortfolioID   = L.PortfolioID
      AND A.Application_ID = L.ApplicationID
WHERE A.ApplicationDate > '2023-04-30'
  AND I.InstallmentNumber = 1;


/* ================================
   STEP 2: FLAG FPDFA + FIRST LOAN INSTALLMENT
   ================================ */
DROP TABLE IF EXISTS #LoanDefault_Flag;

SELECT 
      L.*
      
    /* ---------- FPDFA FLAG ---------- */
    , CASE 
        WHEN L.PaymentStatus = 'R'
         AND L.PaymentType IN ('I','S','A')
         AND L.PaymentMode IN ('A','B','D')
         AND L.DueDate <= CAST(GETDATE() AS date)
         AND NOT EXISTS (
                SELECT 1
                FROM #LF_LoanDefault ld
                WHERE ld.InstallmentID = L.InstallmentID
                  AND ld.PaymentStatus = 'D'
                  AND ld.PaymentType NOT IN ('3','~','Q')
                  AND ld.PaymentMode IN ('A','D','B')
                  AND CONVERT(date, ld.TransactionDate) = CONVERT(date, L.DueDate)
             )
        THEN 1 ELSE 0 
      END AS FPDFA

    /* ---------- FIRST INSTALLMENT FLAG ---------- */
    , CASE 
        WHEN L.LoanStatus NOT IN ('V','W','G','K')
         AND NOT (
              L.iPaymentMode = 144 
              AND L.Pending = 1
              AND L.DueDate >= CAST(GETDATE() AS date)
             )
        THEN 1 ELSE 0 
      END AS is_loan_first_install

INTO #LoanDefault_Flag
FROM #LF_LoanDefault L;


/* ================================
   STEP 3: DEDUP PER APPLICATION
   rn = 1 → prioritize FPDFA DESC
   ================================ */
DROP TABLE IF EXISTS #LoanDefault_Dedup;

WITH dedup AS (
    SELECT 
          LoanID
        , Application_ID
        , APPGUID
        , ApplicationDate
        , ApplicationSteps
        , PortfolioID
        , LoanStatus
        , InstallmentNumber
        , PaymentStatus
        , PaymentType
        , PaymentMode
        , AttemptNo
        , TransactionDate
        , PaymentID
        , InstallmentID
        , iPaymentMode
        , DueDate
        , Status
        , Pending
        , FPDFA
        , is_loan_first_install
        , ROW_NUMBER() OVER (
            PARTITION BY Application_ID, PortfolioID
            ORDER BY FPDFA DESC
          ) AS rn
    FROM #LoanDefault_Flag
)
SELECT *
INTO #LoanDefault_Dedup
FROM dedup
WHERE rn = 1;


/* ================================
   STEP 4: QUICK CHECK
   ================================ */
/* SELECT TOP 10 *
FROM #LoanDefault_Dedup; */


drop table if EXISTS #t3
select A.*, B.FPDFA, B.PaymentStatus
into #t3
from #t2 A 
inner join #LoanDefault_Dedup B on A.LoanID = B.LoanID


/* -------------------------------------------------------------
   After #t3, get FPDAA metrics (#t31)
   ------------------------------------------------------------- */
drop table if EXISTS #t31
select Application_ID, PortFolioID, LoanID, (1-max(FstInstPaidOff)) as FPDAA
into #t31 from
(select A.*, case when P.PaymentStatus = 'D' then 1 else 0 end as FstInstPaidOff, P.PaymentStatus
from #t2 A 
inner join Payment P on A.LoanID = P.LoanID and P.PaymentMode IN ('A','D','B') and PaymentType NOT IN ('3','~','Q')
                        and P.InstallmentNumber=1 and P.AttemptNo >= 1 and P.PaymentDate <= getdate() and P.PaymentStatus in ('D','R','S','B')
where A.Originated=1) K group by Application_ID, PortFolioID, LoanID

-- Peak 10 from #t31
/* SELECT TOP 10 *
FROM #t31 */


/* -------------------------------------------------------------
   After #t31, calculate the paid off amounts --> #t4
   ------------------------------------------------------------- */
drop table if EXISTS #t4
select A.Application_ID, A.PortFolioID, A.LoanID, sum(case when P.PaymentStatus = 'D' then P.PaymentAmount else 0 end) as PaidOffPaymentAmount,
sum(case when P.PaymentStatus = 'R' then 1 else 0 end) as PmtReturn, count(*) as PmtCount
into #t4
from #t2 A 
inner join Payment P on A.LoanID = P.LoanID and P.PaymentMode not in ('V','P','T','H','G') and PaymentType in ('I','S','Z','A','X','Q') 
                        and P.InstallmentNumber >= 1 and P.PaymentDate <= getdate() and P.PaymentStatus in ('D','R') and P.PaymentAmount>=2.95 and (P.PaymentAmount > P.FeeChargePaid)
where A.Originated=1
group by A.Application_ID, A.PortFolioID, A.LoanID
"""
query2 = """
SELECT A.*,B.FPDFA, B2.FPDAA,C.PaidOffPaymentAmount, C.PmtCount, C.PmtReturn,CASE
  WHEN A.OriginatedAmount IS NULL THEN 0                
  WHEN A.OriginatedAmount = 0 THEN 0.0                     
  ELSE COALESCE(C.PaidOffPaymentAmount, 0.0) 
       / CAST(A.OriginatedAmount AS float)
END AS Payin,
case when C.PaidOffPaymentAmount >= A.OriginatedAmount * 1.05 then 1 else 0 end as IsGood
from #t2 A  -- avg(OriginatedAmount+0.0) as AvgLoanAmount, sum(FPDFA) as FPDFA, sum(FPDFA+0.0)/count(*)*100 as FPDFARate
left join #t11 A1 on A1.Application_ID = A.Application_ID and A1.PortFolioID = A.PortfolioID
left join #t3 B on A.Application_ID = B.Application_ID and A.PortFolioID = B.PortFolioID
left join #t31 B2 on A.Application_ID = B2.Application_ID and A.PortFolioID = B2.PortFolioID
left join #t4 C on A.Application_ID = C.Application_ID and A.PortFolioID = C.PortFolioID
where A.Originated = 1
"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_perf = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')
df_perf = df_perf.rename(columns = {'Application_ID':'AppID'})
df_perf.AppID = df_perf.AppID.astype(int)


In [8]:
print(df_perf.shape)
print(empty_model_requests.shape)
print(non_empty_model_requests.shape)





(18099, 33)
(3297, 1)
(154, 1)


In [ ]:
# Minimal right-joins vs df_perf and quick flags
perf_vs_empty = df_perf.merge(empty_model_requests[['IBVStatusID']], on='IBVStatusID', how='right')
perf_vs_nonempty = df_perf.merge(non_empty_model_requests[['IBVStatusID']], on='IBVStatusID', how='right')

score_cols = [c for c in ['IBVScore','NDScore','IBVBand','NDBand'] if c in df_perf.columns]
ext_cols = [c for c in ['B1ExtResponse','B2ExtResponse'] if c in df_perf.columns]

In [12]:
print(perf_vs_empty.shape)

(3316, 36)


In [11]:
for _df in (perf_vs_empty, perf_vs_nonempty):
    _df['has_perf'] = _df['AppID'].notna()
    _df['has_scores'] = _df[score_cols].notna().any(axis=1) if score_cols else False
    _df['has_ext'] = _df[ext_cols].notna().any(axis=1) if ext_cols else False

print({
    'empty_total': len(empty_model_requests),
    'empty_with_perf': int(perf_vs_empty['has_perf'].sum()),
    'empty_with_scores': int(perf_vs_empty['has_scores'].sum()),
    'empty_with_ext': int(perf_vs_empty['has_ext'].sum()),
})
print({
    'nonempty_total': len(non_empty_model_requests),
    'nonempty_with_perf': int(perf_vs_nonempty['has_perf'].sum()),
    'nonempty_with_scores': int(perf_vs_nonempty['has_scores'].sum()),
    'nonempty_with_ext': int(perf_vs_nonempty['has_ext'].sum()),
})

perf_join_empty = perf_vs_empty
perf_join_nonempty = perf_vs_nonempty

{'empty_total': 3297, 'empty_with_perf': 2159, 'empty_with_scores': 2159, 'empty_with_ext': 2159}
{'nonempty_total': 154, 'nonempty_with_perf': 55, 'nonempty_with_scores': 55, 'nonempty_with_ext': 55}


### Analyze failedModelRequest_* JSONs for failure reasons

In [21]:
# Analyze failedModelRequest_* JSONs for failure reasons
import os, re, json
import pandas as pd

input_dir = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_JSONs"
rows = []
parse_errors = []

for fname in os.listdir(input_dir):
    if not (fname.startswith("failedModelRequest_") and fname.endswith(".json")):
        continue
    m = re.search(r"_(\d+)\.json$", fname)
    if not m:
        continue
    ibv_id = int(m.group(1))
    path = os.path.join(input_dir, fname)
    try:
        with open(path, "r", encoding="utf-8") as f:
            payload = json.load(f)
    except Exception as e:
        parse_errors.append((ibv_id, str(e)))
        continue

    accounts = payload.get("accounts")
    accounts = accounts if isinstance(accounts, list) else []
    num_accounts = len(accounts)
    account_guids = {a.get("accountGuid") for a in accounts if isinstance(a, dict) and a.get("accountGuid")}

    transactions = payload.get("transactions")
    transactions = transactions if isinstance(transactions, list) else []
    num_transactions = len(transactions)
    txn_account_guids = {t.get("accountGuid") for t in transactions if isinstance(t, dict) and t.get("accountGuid")}

    accounts_empty = (num_accounts == 0)
    transactions_empty = (num_transactions == 0)
    txn_guid_missing = (not transactions_empty) and (len(txn_account_guids) == 0)
    guid_mismatch = (not transactions_empty) and (len(txn_account_guids) > 0) and (not txn_account_guids.issubset(account_guids))

    # classify a primary reason (ordered)
    if transactions_empty:
        primary_reason = "empty_transactions"
    elif txn_guid_missing:
        primary_reason = "transactions_missing_accountGuid"
    elif guid_mismatch:
        primary_reason = "accountGuid_mismatch"
    elif accounts_empty:
        primary_reason = "accounts_empty"
    else:
        primary_reason = "other"

    rows.append({
        "IBVStatusID": ibv_id,
        "num_accounts": num_accounts,
        "num_transactions": num_transactions,
        "accounts_empty": accounts_empty,
        "transactions_empty": transactions_empty,
        "txn_guid_count": len(txn_account_guids),
        "account_guid_count": len(account_guids),
        "txn_guid_missing": txn_guid_missing,
        "guid_mismatch": guid_mismatch,
        "primary_reason": primary_reason,
    })

failed_reasons_df = pd.DataFrame(rows).sort_values("IBVStatusID").reset_index(drop=True)

print(f"Analyzed files: {len(rows)} | Parse errors: {len(parse_errors)}")
print("Primary reason counts:\n", failed_reasons_df["primary_reason"].value_counts())

# Optional: save summary CSV
summary_csv = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_data/failed_requests_reasons_summary.csv"
os.makedirs(os.path.dirname(summary_csv), exist_ok=True)
failed_reasons_df.to_csv(summary_csv, index=False)
summary_csv


Analyzed files: 153 | Parse errors: 0
Primary reason counts:
 primary_reason
other                   83
empty_transactions      65
accountGuid_mismatch     5
Name: count, dtype: int64


'/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_data/failed_requests_reasons_summary.csv'

In [22]:
# IBVStatusIDs classified as 'other'
assert 'failed_reasons_df' in globals(), "Run the failure reasons analysis cell first"
other_ids = failed_reasons_df.loc[failed_reasons_df['primary_reason'] == 'other', 'IBVStatusID'].sort_values().tolist()
print(f"Count (other): {len(other_ids)}")
other_ids[:50]  # preview first 50



Count (other): 83


[21399,
 23220,
 32413,
 38875,
 40420,
 43212,
 44865,
 44893,
 45027,
 45088,
 45167,
 45568,
 45879,
 46038,
 46212,
 46309,
 46562,
 47371,
 48086,
 48279,
 49123,
 49133,
 49348,
 49692,
 50217,
 51212,
 51297,
 51313,
 51446,
 52325,
 52421,
 52581,
 52629,
 52989,
 53256,
 53683,
 55167,
 55250,
 56314,
 56325,
 56466,
 57418,
 57756,
 57757,
 58312,
 58320,
 58322,
 58472,
 58721,
 59001]